In [12]:
#Importing libraries and data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

models = [
    ('Linear Regression', LinearRegression()),
    ('Lasso Regression', Lasso()),
    ('Ridge Regression', Ridge()),
    ('SVM', SVR()),
    ('Random Forest', RandomForestRegressor()),
    ('Decision Tree', DecisionTreeRegressor()),
    ('Neural Network', MLPRegressor(max_iter=50000)),
    ('XGBoost', XGBRegressor())
]

# models = [
   
#     ('Lasso Regression', Lasso()),
#     ('Random Forest', RandomForestRegressor()),
#     ('XGBoost', XGBRegressor())
# ]

#Importing data
#for 88 samples we are providing the metadata file here
metadata = pd.read_csv('metadata_labeled.csv')
metadata = metadata.drop_duplicates(subset='Run', keep='first')
metadata = metadata.drop('Label',axis=1)
#This is multiqc quality data file
multiqc = pd.read_csv('multiqc_data.csv')
#we are passing the labels file here
label= pd.read_csv('labels.csv',sep='\t')
#Merge metadata and multiqc dataframe on Run
df = pd.merge(metadata, multiqc, on='Run', how='inner')
print("Enter the Machine you are using, like M1 as 1, M2 as 2, M3 as 3...so on")
input_machine = input()
#extract for M0 from label dataframe along with Run column
label= label.iloc[:,[0,int(input_machine)]]
print("You have selected Machine", input_machine)
#rename the last column as label
label.columns = ['Run','Label']
#merge metadata and label dataframe on Run
metadata = pd.merge(label,metadata, on='Run', how='inner')
#Merge df and label dataframe on Run
df = pd.merge(df, label, on='Run', how='inner')
#Merging multiqc and metadata
merged = pd.merge(metadata, multiqc,on='Run')
#Removing columns with no data
merged = merged.dropna(axis=1, how='all')
#Removing columns with only one unique value
merged = merged.loc[:,merged.apply(pd.Series.nunique) != 1]
#Removing columns with more than 40% missing values1
merged = merged.loc[:, merged.isnull().mean() < .4]
#Removing columns with more than 40% zerosz
merged = merged.loc[:, (merged == 0).mean() < .4]
#Remove non-numeric columns
merged = merged.select_dtypes(include=[np.number])
#put Label column at the end in merged dataframe
merged = merged[[c for c in merged if c not in ['Label']] + ['Label']]
merged.head()
#Replace nan with mean of that column
merged = merged.fillna(merged.mean())
df = merged
del input_machine, label, metadata, multiqc, merged

Enter the Machine you are using, like M1 as 1, M2 as 2, M3 as 3...so on
You have selected Machine 5


In [13]:
from sklearn.model_selection import cross_val_score, KFold

def train(X, y):
    # Scale the data
    sc = StandardScaler()
    X = sc.fit_transform(X)

    # Define a function to train a model and calculate metrics using cross-validation
    def train_and_evaluate(model, X, y):
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        mae_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')
        mse_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
        r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')

        mae = -mae_scores.mean()
        mse = -mse_scores.mean()
        r2 = r2_scores.mean()
        

        try:
            model.fit(X, y)
            feature_importance = model.feature_importances_
        except AttributeError:
            feature_importance = None
        
        return mae, mse, r2, feature_importance

    # Define a function to display the metrics in a table
    def display_results(results):
        print(f"{'Model':<20}{'MAE':<10}{'MSE':<10}{'R2':<10}")
        for name, metrics in results.items():
            mae, mse, r2, feature_importance = metrics
            if feature_importance is not None:
                feature_importance = [f"{value:.2f}" for value in feature_importance]
                feature_importance = ' '.join(feature_importance)
            else:
                feature_importance = ''
            print(f"{name:<20}{mae:<10.2f}{mse:<10.2f}{r2:<10.2f}")

    # Train each model, calculate the metrics, and add the results to the table 
    results = {}
    for name, model in models:
        results[name] = train_and_evaluate(model, X, y)

    # Sort the results based on R2 value
    results_sorted = dict(sorted(results.items(), key=lambda item: item[1][2], reverse=True))
    display_results(results_sorted)

In [14]:
# # CASE 1 WITH 385 FEATURES
# df_all_features = df #386-features
# X = df_all_features
# X = X.loc[:,~X.columns.duplicated()]
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# #drop ProjectID column
# X = X.drop('ProjectID',axis=1)
# train(X,y)

In [15]:
# # CASE 2 WITH TOP-10 FEATURES
# # keep the features only in top_features from features_importance
# print("using top-10 features from RF model")
# model = RandomForestRegressor()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# model.fit(X_train, y_train)
# feature_importance = model.feature_importances_
# features = X.columns
# features_importance = dict(zip(features, feature_importance))
# features_importance = dict(sorted(features_importance.items(), key=lambda item: item[1], reverse=True))
# features_importance = dict(list(features_importance.items())[:10])
# features_importance
# features_importance = list(features_importance.keys())
# top_features = X[features_importance]
# top_features['Label'] = y
# X = top_features
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# train(X,y)

In [16]:
df.head()

,spots,bases,spots_with_mates,avgLength,size_MB,InsertSize,ProjectID,None_mqc-generalstats-fastqc-percent_duplicates,None_mqc-generalstats-fastqc-percent_gc,None_mqc-generalstats-fastqc-avg_sequence_length,...,sdl_'>500',sdl_'>1k',fsch_Per Base Sequence Quality,fsch_Per Tile Sequence Quality,fsch_Per Sequence Quality Scores,fsch_Per Base Sequence Content,fsch_Per Sequence GC Content,fsch_Per Base N Content,fsch_Overrepresented Sequences,Label
0,16509214,3565990224,16509214,216,2946,250,33859,5.788338,45.0,108.0,...,0.004571,0.000000,0.25,0.50,0.5,0.25,0.5,1.0,1.0,512
1,16098118,2446913936,16098118,152,1649,175,33847,3.750143,45.0,76.0,...,0.000000,0.012741,1.00,1.00,1.0,0.50,0.5,1.0,1.0,412
2,17697186,2689972272,17697186,152,1888,200,33847,3.715658,43.0,76.0,...,0.003878,0.011432,1.00,1.00,1.0,1.00,0.5,1.0,1.0,379
3,17057394,2592723888,17057394,152,1795,200,33847,3.757012,43.0,76.0,...,0.004317,0.011820,1.00,0.25,1.0,1.00,0.5,1.0,1.0,375
4,16232769,3506278104,0,216,2595,450,33865,6.613267,42.0,108.0,...,0.007455,0.014150,0.25,1.00,1.0,0.50,0.5,1.0,1.0,490


In [17]:
# CASE 3 WITH SIZE_ONLY FEATURE

df_size_only = df.loc[:,['size_MB','Label']] #size_only feature
X = df_size_only
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
size_MB_results = train(X,y)

Model               MAE       MSE       R2        
Neural Network      8.31      183.72    0.75      
Random Forest       9.11      215.72    0.71      
XGBoost             9.65      240.62    0.66      
Ridge Regression    11.62     252.39    0.65      
Linear Regression   11.61     252.54    0.65      
Lasso Regression    11.69     252.95    0.65      
Decision Tree       10.59     299.12    0.58      
SVM                 13.97     327.67    0.53      


In [18]:
# # CASE 4 SELECTED FEATURES ONLY - selected 8 features
# selected_features = ['size_MB', 'bases', 'InsertSize', 'None_mqc-generalstats-fastqc-total_sequences', 'spots','spots_with_mates','Unique Reads','Duplicate Reads']
# selected_features_df = df[selected_features]
# selected_features_df['Label'] = y
# X = selected_features_df
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# train(X,y)

In [19]:
# # Case 5 SELECTED FEATURES ONLY - selected 27 features
# #create a column that contains sum of pbsq of all columns prefix with pbsq and then remove all pbsq columns except the sum column
# pbsq_columns = [col for col in df.columns if 'pbsq' in col]
# pbnc_columns = [col for col in df.columns if 'pbnc' in col]
# psgc_columns = [col for col in df.columns if 'psgc' in col]
# psgcp_columns = [col for col in df.columns if 'psgcp' in col]
# pss_columns = [col for col in df.columns if 'pss' in col]
# sdl_columns = [col for col in df.columns if 'sdl' in col]
# df['sum_pbnc'] = df[pbnc_columns].sum(axis=1)
# df['sum_pbsq'] = df[pbsq_columns].sum(axis=1)
# df['sum_psgc'] = df[psgc_columns].sum(axis=1)
# df['sum_psgcp'] = df[psgcp_columns].sum(axis=1)
# df['sum_pss'] = df[pss_columns].sum(axis=1)
# df['sdl'] = df[sdl_columns].sum(axis=1)
# #remove all columns from df where prefix is pbsq
# df = df.loc[:,~df.columns.str.startswith('pbsq')]
# df = df.loc[:,~df.columns.str.startswith('pbnc')]
# df=df.loc[:,~df.columns.str.startswith('psgc')]
# df=df.loc[:,~df.columns.str.startswith('psgcp')]
# df=df.loc[:,~df.columns.str.startswith('pss')]
# df=df.loc[:,~df.columns.str.startswith('sdl')]  
# #add sum_pbsq column to df
# df['Label'] = y
# X = df
# labelencoder = LabelEncoder()
# X['Label'] = labelencoder.fit_transform(X['Label'])
# y = X['Label']
# X=X.drop('Label', axis=1)
# X=X.drop('ProjectID', axis=1)
# selected_features_results=train(X,y)
# del pbsq_columns, pbnc_columns, psgc_columns, psgcp_columns, pss_columns, sdl_columns


In [20]:
# df.head();df=df.drop('Label',axis=1)
# #Plot correlogram
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,10))
# sns.heatmap(df.corr(), annot=False, cmap='coolwarm', center=0)
# plt.show()

In [21]:
# Case 6 SELECTED FEATURES ONLY - selected xx features
# # With features mentioned in the draft paper
pbsq_columns = [col for col in df.columns if 'pbsq' in col]
pbnc_columns = [col for col in df.columns if 'pbnc' in col]
psgc_columns = [col for col in df.columns if 'psgc' in col]
psgcp_columns = [col for col in df.columns if 'psgcp' in col]
pss_columns = [col for col in df.columns if 'pss' in col]
sdl_columns = [col for col in df.columns if 'sdl' in col]
df['sum_pbnc'] = df[pbnc_columns].sum(axis=1)
df['sum_pbsq'] = df[pbsq_columns].sum(axis=1)
df['sum_psgc'] = df[psgc_columns].sum(axis=1)
df['sum_psgcp'] = df[psgcp_columns].sum(axis=1)
df['sum_pss'] = df[pss_columns].sum(axis=1)
df['sdl'] = df[sdl_columns].sum(axis=1)
#remove all columns from df where prefix is pbsq
df = df.loc[:,~df.columns.str.startswith('pbsq')]
df = df.loc[:,~df.columns.str.startswith('pbnc')]
df=df.loc[:,~df.columns.str.startswith('psgc')]
df=df.loc[:,~df.columns.str.startswith('psgcp')]
df=df.loc[:,~df.columns.str.startswith('pss')]
df=df.loc[:,~df.columns.str.startswith('sdl')]
df.columns
df_2 = df.loc[:,['spots','bases','avgLength','size_MB','InsertSize','None_mqc-generalstats-fastqc-percent_duplicates',
                 'None_mqc-generalstats-fastqc-percent_gc','None_mqc-generalstats-fastqc-avg_sequence_length',
                 'None_mqc-generalstats-fastqc-total_sequences','Unique Reads','fsch_Per Tile Sequence Quality',
                 'fsch_Per Sequence Quality Scores','fsch_Per Base Sequence Content','fsch_Per Sequence GC Content',
                 'fsch_Per Base N Content', 'fsch_Overrepresented Sequences','fsch_Per Base Sequence Quality',
                 'sum_pbnc','sum_pbsq','sum_pss','sum_psgc'
                 ]]
df_2['Label'] = y
X = df_2
labelencoder = LabelEncoder()
X['Label'] = labelencoder.fit_transform(X['Label'])
y = X['Label']
X=X.drop('Label', axis=1)
train(X,y)

Model               MAE       MSE       R2        
Random Forest       8.61      190.42    0.75      
Lasso Regression    10.94     238.25    0.67      
Ridge Regression    10.49     238.59    0.67      
XGBoost             9.69      266.26    0.63      
Linear Regression   11.02     279.35    0.62      
Decision Tree       11.04     387.85    0.48      
SVM                 20.96     604.64    0.13      
Neural Network      24.37     975.87    -0.73     
